In [1]:

import json
import os
import sys
sys.path.append('../')
from my_packages.data_processing.attributes_processing import transform_test_list
from my_packages.data_processing.split_dataset import read_dataset_to_json

main_dataset_folder = '../data/mbpp_transformed_code_examples/sanitized-MBPP-midio.json'
dataset = read_dataset_to_json(main_dataset_folder)

for sample in dataset:
    sample['external_functions'] = sample['library_functions'].copy()
    sample.pop('library_functions')

    new_test_list = transform_test_list(sample['testing']['test_list'])
    sample['testing']['test_list'] = new_test_list
    sample['testing']['external_functions'] = sample['testing']['library_functions'].copy()
    sample['testing'].pop('library_functions')
    


print(json.dumps(dataset[0], indent=4))
current_dir = os.getcwd()  # Gets the current working directory
full_path = os.path.join(current_dir, '../data')

os.makedirs(full_path, exist_ok=True)

with open(f'{full_path}/MBPP-Midio-50.json', 'w') as train_file:
    json.dump(dataset, train_file, indent=4)

{
    "prompts": [
        "Create a function that checks whether the given two integers have opposite sign or not.",
        "The flow should create a user-defined function. The body of the function contains two input properties of type Number, a 'Math Expression' node and one output property of type 'Bool'.\n The arrows are connected as follows: The input properties is connected with arrows to 'Math Expression', that contains an expression with the two input properties of type numbers. The 'Math Expression' result is a boolean. This value is connected with arrows to the created output property, that return the Bool from the function. The user-defined function is then created within the main module.",
        "This created user-defined function should then be tested within a seperate module called tests, within the main module. Since the user-defined function does not contain input and ouput triggers, the flow of execution is sent between the assert nodes, using their input and output